In [1]:
## setup
!pip install ../input/nnaudio031/nnAudio-0.3.1-py3-none-any.whl

Processing /kaggle/input/nnaudio031/nnAudio-0.3.1-py3-none-any.whl


In [2]:
# !ls /kaggle/input/blef0503/
# !ls /kaggle/input/ef-v0/
# !ls /kaggle/input/birds-inference-pip-wheels/*
# !pip freeze|grep torch

In [3]:
## https://github.com/Selimonder/birdclef2022/
import os, sys, glob, math

os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

github_folder = "/kaggle/input/blef0508v3"
sys.path.append(github_folder)

In [4]:
import cv2
import gc
import time
import torch
import librosa
import argparse, warnings

import numpy as np
import pandas as pd
import IPython.display as ipd
from pathlib import Path


import zoo
from training.config import load_config
from training.datasets import BirdDatasetOOF


cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(0)

from tqdm import tqdm
from torch.utils.data import DataLoader

warnings.simplefilter("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


## models

In [5]:
def load_model(conf_path: str, weights_path: str, prefix: str, suffix: str, fold: int, to_device: bool = True):
    conf = load_config(conf_path)
    conf['encoder_params']['pretrained'] = False
    print(conf['encoder_params']['pretrained'], type(conf['encoder_params']['pretrained']))
    
    snapshot_name = "{}{}_{}_{}_{}".format(prefix, conf["network"], conf["encoder_params"]["encoder"], fold, suffix)
    weights_path = os.path.join(weights_path, snapshot_name)
    print(weights_path)
    
    model = zoo.__dict__[conf["network"]](**conf["encoder_params"])
    model = torch.nn.DataParallel(model)
    print("=> loading checkpoint '{}''".format(weights_path))
    checkpoint = torch.load(weights_path, map_location="cpu")
    print("epoch", checkpoint["epoch"])
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    if to_device: model.cuda()
    return model

models = []

## best only for now
suffixes = ["lb"]#, "f1_score" "last"]
folds    = 5

# 16min rank82  cls_ef_v0.json weights_path = f"/kaggle/input/cls-ef-v0", prefix = "ef_v0",
# rank90  cls_ef_v0.json weights_path = f"/kaggle/input/ef-v0-pretrain", prefix = "ef_v0_",

# 44 min0.79 240 conf_path    = f"{github_folder}/configs/cls_nf0_v5.json", weights_path = f"/kaggle/input/nf0-v5", prefix       = "nf0_v5_",

# 
#  cls_nf0_v5.json",weights_path = nfnet-baseline-bs16/nfnet-baseline-bs16, prefix= "nf0_v5_",


# 0.78
# f"{github_folder}/configs/cls_nf0_v5.json",
# weights_path = f"/kaggle/input/nfnet-baseline-bs16/nfnet-baseline-bs16",
# prefix       = "nf0_v5_",


# 

for i in [0]:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
                               weights_path = f"/kaggle/input/e40-ef0",
                               prefix       = "ef_v0",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

len(models)
#models = [models[0]]

False <class 'bool'>
/kaggle/input/e40-ef0/ef_v0TimmClassifier_v3_tf_efficientnet_b0_0_lb
initing CLS features model 15 duration...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint '/kaggle/input/e40-ef0/ef_v0TimmClassifier_v3_tf_efficientnet_b0_0_lb''
epoch 19


1

In [6]:
import torch

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(models[0])//1024

3593

## infer

In [7]:
CLASSES = "abethr1 abhori1 abythr1 afbfly1 afdfly1 afecuc1 affeag1 afgfly1 afghor1 afmdov1 afpfly1 afpkin1 afpwag1 afrgos1 afrgrp1 afrjac1 afrthr1 amesun2 augbuz1 bagwea1 barswa bawhor2 bawman1 bcbeat1 beasun2 bkctch1 bkfruw1 blacra1 blacuc1 blakit1 blaplo1 blbpuf2 blcapa2 blfbus1 blhgon1 blhher1 blksaw1 blnmou1 blnwea1 bltapa1 bltbar1 bltori1 blwlap1 brcale1 brcsta1 brctch1 brcwea1 brican1 brobab1 broman1 brosun1 brrwhe3 brtcha1 brubru1 brwwar1 bswdov1 btweye2 bubwar2 butapa1 cabgre1 carcha1 carwoo1 categr ccbeat1 chespa1 chewea1 chibat1 chtapa3 chucis1 cibwar1 cohmar1 colsun2 combul2 combuz1 comsan crefra2 crheag1 crohor1 darbar1 darter3 didcuc1 dotbar1 dutdov1 easmog1 eaywag1 edcsun3 egygoo equaka1 eswdov1 eubeat1 fatrav1 fatwid1 fislov1 fotdro5 gabgos2 gargan gbesta1 gnbcam2 gnhsun1 gobbun1 gobsta5 gobwea1 golher1 grbcam1 grccra1 grecor greegr grewoo2 grwpyt1 gryapa1 grywrw1 gybfis1 gycwar3 gyhbus1 gyhkin1 gyhneg1 gyhspa1 gytbar1 hadibi1 hamerk1 hartur1 helgui hipbab1 hoopoe huncis1 hunsun2 joygre1 kerspa2 klacuc1 kvbsun1 laudov1 lawgol lesmaw1 lessts1 libeat1 litegr litswi1 litwea1 loceag1 lotcor1 lotlap1 luebus1 mabeat1 macshr1 malkin1 marsto1 marsun2 mcptit1 meypar1 moccha1 mouwag1 ndcsun2 nobfly1 norbro1 norcro1 norfis1 norpuf1 nubwoo1 pabspa1 palfly2 palpri1 piecro1 piekin1 pitwhy purgre2 pygbat1 quailf1 ratcis1 raybar1 rbsrob1 rebfir2 rebhor1 reboxp1 reccor reccuc1 reedov1 refbar2 refcro1 reftin1 refwar2 rehblu1 rehwea1 reisee2 rerswa1 rewsta1 rindov rocmar2 rostur1 ruegls1 rufcha2 sacibi2 sccsun2 scrcha1 scthon1 shesta1 sichor1 sincis1 slbgre1 slcbou1 sltnig1 sobfly1 somgre1 somtit4 soucit1 soufis1 spemou2 spepig1 spewea1 spfbar1 spfwea1 spmthr1 spwlap1 squher1 strher strsee1 stusta1 subbus1 supsta1 tacsun1 tafpri1 tamdov1 thrnig1 trobou1 varsun2 vibsta2 vilwea1 vimwea1 walsta1 wbgbir1 wbrcha2 wbswea1 wfbeat1 whbcan1 whbcou1 whbcro2 whbtit5 whbwea1 whbwhe3 whcpri2 whctur2 wheslf1 whhsaw1 whihel1 whrshr1 witswa1 wlwwar wookin1 woosan wtbeat1 yebapa1 yebbar1 yebduc1 yebere1 yebgre1 yebsto1 yeccan1 yefcan yelbis1 yenspu1 yertin1 yesbar1 yespet1 yetgre1 yewgre1".split()
N_CLASS = len(CLASSES)

In [8]:
from torch.utils.data import Dataset, DataLoader

frame_length = 5
infer_frame_length = 5
sample_rate  = 32000

class TestDataset(Dataset):
    def __init__(self, 
                 df: pd.DataFrame, 
                 sample_rate:int,
                 frame_length:int,
                 waveform
                ):
        
        self.df = df
        
        self.frame_length = frame_length
        self.sample_rate = sample_rate
        
        self.waveform = waveform
        self.n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))

    def __len__(self):
        return self.n_parts

    def __getitem__(self, idx: int):
        sample = self.df.loc[idx, :]
        row_id = sample.row_id
        sample_rate = self.sample_rate
        frame_length = self.frame_length

        end_seconds = int(sample.seconds)
        start_seconds = int(end_seconds - 5)
        
        y = self.waveform[sample_rate * start_seconds : sample_rate * end_seconds].astype(np.float32)
        
        wav_tensor = torch.from_numpy(y)
        if len(wav_tensor) == frame_length * sample_rate:
            pass
        elif len(wav_tensor) < frame_length * sample_rate:
            wav_tensor = torch.nn.functional.pad(wav_tensor, (0, (frame_length * sample_rate) - len(wav_tensor)))
        elif len(wav_tensor) > frame_length * sample_rate:
            wav_tensor = wav_tensor[:(frame_length * sample_rate)]
        
        # add channel
        wav_tensor = wav_tensor.unsqueeze(0)
        
        return {
            "wav_tensors": wav_tensor,
            "row_id": row_id,
        }


In [9]:
all_audios = list(Path("../input/birdclef-2023/test_soundscapes/").glob("*.ogg")) 
seconds = [i for i in range(5, 605, 5)]

class CFG:
    batch_size=4
    num_workers=4
    
config = CFG()

def prediction_for_clip(
    audio_path
):
    
    device = torch.device("cpu")
    
    # inference
    prediction_dict = {}
    
    clip, _ = librosa.load(audio_path, sr=sample_rate, mono=True)
    name_ = "_".join(audio_path.name.split(".")[:-1])
    row_ids = [name_+f"_{second}" for second in seconds]

    test_df = pd.DataFrame({
        "row_id": row_ids,
        "seconds": seconds
    })
    
        
    dataset = TestDataset(
        df=test_df, 
        sample_rate=sample_rate,
        frame_length=frame_length,
        waveform=clip
    )

    loader = DataLoader(
        dataset,
        batch_size=config.batch_size, 
        num_workers=config.num_workers,
        drop_last=False,
        shuffle=False,
        pin_memory=True
    )

    for data in loader:

        row_ids = data['row_id']

        for row_id in row_ids:
            if row_id not in prediction_dict:
                prediction_dict[str(row_id)] = []

        probas = []

        for model in models:
            with torch.no_grad():
                # print(type(data["wav_tensors"]))
                # print(data["wav_tensors"].size())
                # output = model(data["wav_tensors"])
                output = model(data["wav_tensors"], is_test=True)["logit"].sigmoid()
#                     
            for row_id_idx, row_id in enumerate(row_ids):
                prediction_dict[str(row_id)].append(output[[row_id_idx]].numpy().reshape(-1))
                                                        
    for row_id in list(prediction_dict.keys()):
                
        logits = np.array(prediction_dict[row_id]).mean(0)
        prediction_dict[row_id] = {}
        for label in range(N_CLASS):
            prediction_dict[row_id][CLASSES[label]] = logits[label]

    return prediction_dict

In [10]:
# %%time
# start = time.time()
# dicts = []
# for audio_path in all_audios:
#     dicts.append(prediction_for_clip(audio_path))
# print(f"Regular for loop costs {time.time()-start} for processing {len(all_audios)} audios")

In [11]:
import concurrent.futures

start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    dicts = list(executor.map(prediction_for_clip, all_audios))
print(f"With concurrent ThreadPoolExecutor, time cost reduced to {time.time()-start:.1f} for processing {len(all_audios)} audios")


With concurrent ThreadPoolExecutor, time cost reduced to 19.3 for processing 1 audios


In [12]:
prediction_dicts = {}
for d in dicts:
    prediction_dicts.update(d)
    
submission = pd.DataFrame.from_dict(prediction_dicts, "index").rename_axis("row_id").reset_index()
submission.to_csv("submission.csv", index=False)
print(f"""
total rows : {len(submission)}
""")
submission.head()


total rows : 120



,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.024468,0.016309,0.128915,0.002223,0.002169,0.008502,0.007558,0.004082,0.025380,...,0.002207,0.005911,0.013596,0.002669,0.016673,0.006696,0.008719,0.003567,0.005940,0.007900
1,soundscape_29201_10,0.018099,0.153154,0.006084,0.049784,0.009905,0.013888,0.020096,0.003213,0.013634,...,0.001162,0.005886,0.013432,0.009267,0.034829,0.029104,0.020500,0.006882,0.010174,0.071354
2,soundscape_29201_15,0.020476,0.023540,0.034404,0.018561,0.007867,0.039497,0.011572,0.009824,0.009682,...,0.003252,0.008721,0.011465,0.009803,0.012738,0.037714,0.010992,0.013150,0.013500,0.035175
3,soundscape_29201_20,0.004694,0.018199,0.011695,0.010560,0.013242,0.009848,0.014674,0.009491,0.006400,...,0.004020,0.006802,0.017739,0.007555,0.027206,0.010791,0.022323,0.008253,0.017065,0.009040
4,soundscape_29201_25,0.002801,0.015591,0.010036,0.014981,0.013096,0.007631,0.005348,0.006919,0.006193,...,0.000730,0.003299,0.009305,0.008281,0.013102,0.012779,0.012167,0.005364,0.006497,0.009672


## my infer

In [13]:
# from torch.utils.data import Dataset, DataLoader

# class BirdDataset(Dataset):
#     def __init__(self, frame_length, sample_rate):
#         self.frame_length = frame_length
#         self.sample_rate = sample_rate
#         self.data   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg")) 
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         """
#         Prepare audio clip for inference
#         """
#         fpath = self.data[idx]
#         sample_rate = self.sample_rate
#         frame_length = self.frame_length
        
#         infer_frame_length = frame_length
#         batch = {"wav_tensors": [], "end_times": []}

#         waveform, sample_rate = librosa.load(fpath, sr=sample_rate, mono=True)
#         n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))

#         for seg_idx in range(n_parts): 
#             end_time = (seg_idx + 1) * frame_length
#             seg_wav  = waveform[(end_time*sample_rate)-(sample_rate*frame_length):end_time*sample_rate]

#             wav_tensor = torch.from_numpy(seg_wav)

#             if len(wav_tensor) == frame_length * sample_rate:
#                 batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
#             elif len(wav_tensor) < frame_length * sample_rate:
#                 wav_tensor = torch.nn.functional.pad(wav_tensor, (0, (frame_length * sample_rate) - len(wav_tensor)))
#                 batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
#             elif len(wav_tensor) > frame_length * sample_rate:
#                 wav_tensor = wav_tensor[:(frame_length * sample_rate)]
#                 batch["wav_tensors"].append(wav_tensor.unsqueeze(0))

#             batch["end_times"].append(end_time)
#         batch["wav_tensors"] = torch.stack(batch["wav_tensors"])
#         return batch, n_parts, len(waveform)

# @torch.no_grad()
# def predict_clip(models, batch, n_parts, frame_length):
#     preds = np.zeros([len(models), n_parts, N_CLASS])
#     for m_idx, model in enumerate(models):
#         with torch.cuda.amp.autocast():
#             preds[m_idx] = model(batch["wav_tensors"], is_test=True)["logit"].sigmoid().cpu().numpy()
#     return preds.mean(0) ## max by model    




In [14]:
# frame_length = 5
# infer_frame_length = 5
# sample_rate  = 32000
# test_ds = BirdDataset(frame_length, sample_rate)

# # loader = DataLoader(
# #     test_ds,
# #     batch_size=16, 
# #     num_workers=4,
# #     drop_last=False,
# #     shuffle=False,
# #     pin_memory=True
# # )

# len(models), len(test_ds)

In [15]:
# %%time

# @torch.no_grad()
# def generate_preds():
#     debug = False
#     preds        = []
#     # test_files   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg")) 
#     scored_birds = np.array(CLASSES)
#     # print("test file num:", len(test_files))
    
#     for batch, n_parts, len_waveform in tqdm(test_ds):
#         # file_id = os.path.basename(fpath).replace(".ogg", "")
#         # batch, n_parts, len_waveform = prepare_clip(fpath, frame_length, sample_rate)
#         clip_preds = predict_clip(models, batch, n_parts, frame_length)
#         preds.append(clip_preds)
        
#         ## switch to infer frame length
#         n_parts_sub = math.ceil(len_waveform / int(infer_frame_length * sample_rate))
#         clip_preds = np.array_split(clip_preds, n_parts_sub, axis=0)
        
                
#     prob_array = np.array(preds)
    
#     scored_bird_name2idx = {}
#     for i, x in enumerate(scored_birds):
#         scored_bird_name2idx[x] = i

#     return prob_array, scored_bird_name2idx

# prob_array, scored_bird_name2idx = generate_preds()
# # del models
# # gc.collect()
# # torch.cuda.empty_cache()
# prob_array.shape

----

In [16]:
# def generate_file_ids():
#     test_files   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg"))
    
#     file_ids = []
    
#     for fpath in test_files:
#         file_id = os.path.basename(fpath).replace(".ogg", "")
#         file_ids.append(file_id)
        
#     return file_ids

# file_ids = generate_file_ids()

In [17]:
# ## fix aniani
# avg_preds = prob_array

# avg_preds.shape

In [18]:
# submission   = []
# infer_frame_length = 5

# for clip_preds, file_id in zip(avg_preds, file_ids):
#     for frame_idx, pred in enumerate(clip_preds):
#         end_time = (frame_idx + 1) * infer_frame_length
#         d = {
#             "row_id": f"{file_id}_{end_time}",
#             # "target": pred[bi] > threshold,
#         }
#         for bi, bird in enumerate(CLASSES):
#             d[bird] = pred[bi]
#         submission.append(d)

In [19]:
# %cd /kaggle/working

In [20]:
# df_submission = pd.DataFrame(submission).set_index("row_id")
# df_submission.to_csv("submission.csv")

In [21]:
# print(f"""
# total rows : {len(df_submission)}
# """)

# df_submission.head()